In [1]:
'''
About Dataset
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. 
The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

Case Study Excercise
- clean ecommerce data
- summarise sale figures for 2011
- create a dashboard to present sales figures, with the following questions in mind:
    1. What is our order fulfilment, orders vs cancellation?
    2. Who are our main customers?
    3. Which area of the business to focus for the next year?
    4. Devise a marketing strategy to increase sales

source : https://www.kaggle.com/datasets/aliessamali/ecommerce
'''

'\nAbout Dataset\nThis is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. \nThe company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.\n\nCase Study Excercise\n- clean ecommerce data\n- summarise sale figures for 2011\n- create a dashboard to present sales figures, with the following questions in mind:\n    1. What is our order fulfilment, orders vs cancellation?\n    2. Who are our main customers?\n    3. Which area of the business to focus for the next year?\n    4. Devise a marketing strategy to increase sales\n\nsource : https://www.kaggle.com/datasets/aliessamali/ecommerce\n'

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
# Load file into jupyter notebook
path = r"C:\Users\Chris\OneDrive\Desktop\Python Projects"
data = pd.read_csv(path + '\\dataset.csv')
print(data)

       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

                InvoiceDate  UnitPrice  CustomerID         Country  
0     

In [4]:
df = data.copy()
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [5]:
'''
InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
Description: Product (item) name. Nominal.
Quantity: The quantities of each product (item) per transaction. Numeric.
InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
UnitPrice: Unit price. Numeric, Product price per unit in sterling.
CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
Country: Country name. Nominal, the name of the country where each customer resides.
'''

# Validata clean data and count number of orders
cancel = []
orders = []
others = []
cancel_pattern = r'c\d{6}'
order_pattern = r'\d{6}'
for ind in df['InvoiceNo']:
    if re.search(cancel_pattern, ind, flags = re.IGNORECASE):
        cancel.append(ind)
    if re.search(order_pattern, ind):
        orders.append(ind)
    else:
        others.append(ind)
print('cancelled orders:', len(cancel), '\n', 'orders:', len(orders), '\n', 'others:', len(others))
# catch-all variable 'others' to ensure only valid invoice numbers

cancelled orders: 9288 
 orders: 541909 
 others: 0


In [6]:
# Search for non sales related data. 
# Utilised stock code to isolate for 5 digit integers for sales related items; place other stock codes into list  
other_stock = []
stock_pattern = r'\d{5}'
for ind in df['StockCode']:
    if re.search(stock_pattern, ind):
        continue
    else:
        if ind not in other_stock:
            other_stock.append(ind)
        else:
            continue
print(other_stock)


['POST', 'D', 'C2', 'DOT', 'M', 'BANK CHARGES', 'S', 'AMAZONFEE', 'DCGS0076', 'DCGS0003', 'gift_0001_40', 'DCGS0070', 'm', 'gift_0001_50', 'gift_0001_30', 'gift_0001_20', 'DCGS0055', 'DCGS0072', 'DCGS0074', 'DCGS0069', 'DCGS0057', 'DCGSSBOY', 'DCGSSGIRL', 'gift_0001_10', 'PADS', 'DCGS0004', 'DCGS0073', 'DCGS0071', 'DCGS0068', 'DCGS0067', 'DCGS0066P', 'B', 'CRUK']


In [7]:
# remove non sales related stock codes
print('previous count', '\n', df.count(), '\n')
clean_df = df[~df['StockCode'].isin(other_stock)]
clean_df.reset_index(drop = True, inplace = True)
print('current count', '\n', clean_df.count())

previous count 
 InvoiceNo      541909
StockCode      541909
Description    540455
Quantity       541909
InvoiceDate    541909
UnitPrice      541909
CustomerID     406829
Country        541909
dtype: int64 

current count 
 InvoiceNo      538914
StockCode      538914
Description    537475
Quantity       538914
InvoiceDate    538914
UnitPrice      538914
CustomerID     404909
Country        538914
dtype: int64


In [8]:
# ignore customerid nan values because it is not a unique identifier. 
# Althought the description column presents with some nan values, it is less of a concern as it would not affect our sales figures.
clean_df = clean_df.drop(columns = 'CustomerID')
print(clean_df)

       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
538909    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
538910    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
538911    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
538912    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
538913    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

                InvoiceDate  UnitPrice         Country  
0       2010-12-01

In [9]:
# create new column called order status to categorise cancelled vs fulfilled orders
clean_df['order_status'] = np.where(clean_df['InvoiceNo'].str.match(cancel_pattern, flags = re.IGNORECASE), 'cancelled', 'order')
print(clean_df['order_status'].unique())

['order' 'cancelled']


In [10]:
# replacing c wih nothing to get an integer only invoice number
clean_df['InvoiceNo'] = clean_df['InvoiceNo'].str.replace(r'^C', '', flags = re.IGNORECASE, regex = True)

In [11]:
# remove unfulfilled orders where unitprice = 0
print(clean_df[clean_df['UnitPrice'] == 0])
clean_df = clean_df[~(clean_df['UnitPrice'] == 0)]

       InvoiceNo StockCode                   Description  Quantity  \
619       536414     22139                           NaN        56   
1964      536545     21134                           NaN         1   
1965      536546     22145                           NaN         1   
1966      536547     37509                           NaN         1   
1981      536549    85226A                           NaN         1   
...          ...       ...                           ...       ...   
533999    581234     72817                           NaN        27   
535518    581406    46000M  POLYESTER FILLER PAD 45x45cm       240   
535519    581406    46000S  POLYESTER FILLER PAD 40x40cm       300   
535568    581408     85175                           NaN        20   
535933    581422     23169                       smashed      -235   

                InvoiceDate  UnitPrice         Country order_status  
619     2010-12-01 11:52:00        0.0  United Kingdom        order  
1964    2010-12-01 

In [12]:
# add revenue column and reorganise column order
clean_df['revenue'] = clean_df['UnitPrice'] * clean_df['Quantity']
clean_df = clean_df[['InvoiceNo', 'StockCode', 'Description', 'InvoiceDate', 'Quantity', 'UnitPrice', 'revenue', 'Country', 'order_status']]
print(clean_df)

       InvoiceNo StockCode                          Description  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER   
1         536365     71053                  WHITE METAL LANTERN   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.   
...          ...       ...                                  ...   
538909    581587     22613          PACK OF 20 SPACEBOY NAPKINS   
538910    581587     22899         CHILDREN'S APRON DOLLY GIRL    
538911    581587     23254        CHILDRENS CUTLERY DOLLY GIRL    
538912    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE   
538913    581587     22138        BAKING SET 9 PIECE RETROSPOT    

                InvoiceDate  Quantity  UnitPrice  revenue         Country  \
0       2010-12-01 08:26:00         6       2.55    15.30  United Kingdom   
1       2010-12-01 08:26:00         6    

In [13]:
# change data types for data analysis
clean_df['InvoiceNo'] = clean_df['InvoiceNo'].astype('int64')
clean_df['InvoiceDate'] = clean_df['InvoiceDate'].astype('datetime64[ns]')
clean_df['revenue'] = clean_df['revenue'].astype('float64')
clean_df.reset_index()
print(clean_df.dtypes)

InvoiceNo                int64
StockCode               object
Description             object
InvoiceDate     datetime64[ns]
Quantity                 int64
UnitPrice              float64
revenue                float64
Country                 object
order_status            object
dtype: object


In [14]:
# export dataframe to csv for analysis
chart = r'C:\Users\Chris\OneDrive\Desktop\Tableu'
clean_df.to_csv(chart + '\\ecommerce_v4.csv', index = False)